Aging Provision Policy – Additional Provision Calculation 

Policy Overview
To ensure prudent financial coverage, additional provisions are calculated for inventory in special categories, using the following rules:

1. Damaged Inventory
Provision Rate:
100% of the net total cost of damaged stock.
2. Leftover Inventory
For Closed Brands:
50% of the net total cost of leftover stock from brands that are closed.
For Running Brands:
15% of the net total cost of leftover stock from brands that are still operating.
3. Closed Brands (Other Locations)
Provision Rate:
50% of the net total cost of stock from closed brands (excluding leftover and damaged locations).
4. Exclusions
No additional provision is made for inventory under consignment, guaranteed margin, or “Buying Pull - Mango” models.
Summary:
These additional provisions are applied on top of the standard aging policy to ensure adequate financial coverage for higher-risk inventory categories.

1. update the closed brands status in the mapping file
2. check the totals of the net cost throughout the steps to ensure the completness
3. update the combinations in the combination file
4. check missing standard brands in the mapping file and soh
5. check std season and buckets in the checks section of the code
6. update the parameters below
7. check for garbage std season in the code
8. ensure no duplicates in the combination[std brand name and location name] and mapping file [original brand name]
 

In [68]:
import pandas as pd
import numpy as np
import os
pd.options.display.float_format = '{:,.2f}'.format

parameters

In [69]:

SOH_File = 'SOH-mar.xlsx'
first_first_bucket_number_seasons = 5
damage_percentage = 1
leftover_running_percentage = .15
leftover_closed_percentage = .5
closed_percentage = .5

brand_specific_provision = {}
'''    'FLYING TIGER': 2
}'''

"    'FLYING TIGER': 2\n}"

In [70]:
import re

def season_sort_key(season):
    if not isinstance(season, str) or len(season) < 4:
        return (0, 0)  # Push unknowns to bottom

    season_type = season[:2]  # 'AW' or 'SS'
    year = int(season[-2:])

    # AW should come *before* SS, so give it a higher priority
    season_rank = 1 if season_type == "AW" else 0

    return (year, season_rank)

def standardize_season(raw_season):
    if not isinstance(raw_season, str) or raw_season.strip() == "":
        return "Unknown"

    season = raw_season.strip().upper()

    # --- Priority: continuity/old ---
    if "CONTINUITY" in season or "BASICS" in season:
        return "Continuity"

    elif "OLD" in season:
        return "Old-"

    # --- 4-digit year + seasonal tag ---
    year_match = re.search(r"(20\d{2})", season)
    if year_match:
        year = year_match.group(1)[2:]  # Take last 2 digits, e.g., '2023' → '23'

        if any(tag in season for tag in ["SPRING", "SUMMER", "SS"]):
            return f"SS{year}"
        elif any(tag in season for tag in ["AUTUMN", "WINTER", "AW"]):
            return f"AW{year}"

    # --- 2-digit season pattern like AW23, SS22 ---
    match = re.search(r"(SS|AW)(\d{2})", season)
    if match:
        return f"{match.group(1)}{match.group(2)}"

    # --- WA fallback to AW ---
    if "WA" in season:
        match = re.search(r"WA(\d{2})", season)
        if match:
            return f"AW{match.group(1)}"

    # --- Last resort: any 2-digit year with best guess ---
    match = re.search(r"(\d{2})", season)
    if match:
        return f"SS{match.group(1)}"
    
    return "Unknown"  # default if nothing matches


aging provision as per policy

In [71]:
soh = pd.read_excel(fr"SOH/{SOH_File}", sheet_name='Sheet1')
soh = soh[(soh['GROUP_NAME'] != 'Aleph' )& (soh['AR Comments'] == 'Consider')]
existing_balances = pd.read_excel(r'Existing Balances/current_balance.xlsx', sheet_name='Sheet1')
mapping = pd.read_excel(r'Mapping & Combinations/mapping.xlsx', sheet_name='Sheet1')
soh['NETTOTAL_COST'].fillna(0, inplace=True)
soh['NETTOTAL_COST'] = pd.to_numeric(soh['NETTOTAL_COST'], errors='coerce')

original_season = 'SEASON_DESC' if 'SEASON_DESC' in soh.columns else 'SEASON DESC'
soh['NETTOTAL_COST'].sum()


C:\Users\saad.qureshi\AppData\Local\Temp\ipykernel_8768\1067738966.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  soh['NETTOTAL_COST'].fillna(0, inplace=True)


418141106.11822

In [72]:
combinations = pd.read_excel(r'Mapping & Combinations/combinations.xlsx', sheet_name='Sheet1')

In [73]:
soh['GROUP_NAME'] = soh['GROUP_NAME'].str.upper()
mapping['GROUP_NAME'] = mapping['GROUP_NAME'].str.upper()
soh = soh.merge(mapping, on='GROUP_NAME', how='left')
#soh['Std Brand'] = np.where(soh['Std Brand'].isna(), soh['GROUP_NAME'], soh['Std Brand'])
soh = soh[(soh['Closed_status'] != 'Exit' )]
soh['NETTOTAL_COST'].sum()

418141106.11822

In [74]:
'''valid_brands = soh.groupby('Std Brand')['NETTOTAL_COST'].sum()
valid_brands = valid_brands[valid_brands != 0].index

# Step 2: filter original DataFrame
soh = soh[soh['Std Brand'].isin(valid_brands)]
soh['NETTOTAL_COST'].sum()'''

"valid_brands = soh.groupby('Std Brand')['NETTOTAL_COST'].sum()\nvalid_brands = valid_brands[valid_brands != 0].index\n\n# Step 2: filter original DataFrame\nsoh = soh[soh['Std Brand'].isin(valid_brands)]\nsoh['NETTOTAL_COST'].sum()"

In [75]:
soh['std_season'] = soh[original_season].apply(standardize_season)

In [76]:
excluded = {'Unknown', 'Continuity', 'Old-','AW97'}
unique_season = [f for f in soh['std_season'].dropna().unique() if f not in excluded]

In [77]:
sorted_std_season_in_soh = sorted(unique_season, key=season_sort_key, reverse=True)

In [78]:
bucket1 = sorted_std_season_in_soh[:first_first_bucket_number_seasons]
bucket2 = sorted_std_season_in_soh[first_first_bucket_number_seasons:first_first_bucket_number_seasons + 3]
bucket3 = sorted_std_season_in_soh[first_first_bucket_number_seasons+3:first_first_bucket_number_seasons + 6] 
bucket4 = sorted_std_season_in_soh[first_first_bucket_number_seasons + 6:]

In [79]:
bucket1

['AW25', 'SS25', 'AW24', 'SS24', 'AW23']

In [80]:
bucket1 =   bucket1 + ['Unknown','Continuity'] 
bucket4 = bucket4 + [  'Old-','AW97'] 

In [81]:

conditions = [
    soh['std_season'].isin(bucket1),
    soh['std_season'].isin(bucket2),
    soh['std_season'].isin(bucket3)
]

choices = ['bucket1', 'bucket2', 'bucket3']

soh['season_bucket'] = np.select(conditions, choices, default='bucket4')

In [82]:
soh['Continuity_factor'] = 0.40
soh['provision_%_policy'] = soh['season_bucket'].map({
    'bucket1': 0,
    'bucket2': 0.15,
    'bucket3': 0.50,
    'bucket4': 0.75  # Assuming bucket4 is the default for other buckets

})  # Or any default value for other buckets like 'bucket3', 'bucket4'

In [83]:

soh.loc[
    soh['Model'].isin([
        'Consignment',
        'Guaranteed Margin',
        'Buying Pull - Mango'
    ]),
    ['provision_amount_policy', 'provision_%_policy', 'Continuity_factor']
] = 0

soh['provision_amount_policy'] = soh['NETTOTAL_COST'] * soh['provision_%_policy'] * soh['Continuity_factor']

Damage locations


In [84]:
soh['location_catergory'] = "Store, Online & WH"

soh.loc[
    soh['LOCATION_NAME'].astype(str).str.lower().str.contains('leftover', na=False),
    "location_catergory"
] = "Leftover"
soh.loc[soh['LOCATION_NAME'].astype(str).str.lower().str.contains('damage', na=False),"location_catergory"] = "Damage"
soh.loc[soh['LOCATION_NAME'].astype(str).str.lower().str.contains('sulay', na=False),"location_catergory"] = "Leftover"

In [85]:
condition = soh['location_catergory'] == "Damage"  


soh.loc[condition, 'additional_provision'] = (soh["NETTOTAL_COST"] * damage_percentage) - soh['provision_amount_policy']



Leftover

In [86]:

#Leftover of the closed brands
condition =(
    (soh['Closed_status'] == "Closed") & 
    ( soh['location_catergory'] == "Leftover" ) 
    ) 

soh.loc[condition, 'additional_provision'] = (soh["NETTOTAL_COST"] * leftover_closed_percentage) - soh['provision_amount_policy']

condition =(
    (soh['Closed_status'] != "Closed") & 
    ( soh['location_catergory'] == "Leftover" ) 
    ) 

soh.loc[condition, 'additional_provision'] = (soh["NETTOTAL_COST"] * leftover_running_percentage) - soh['provision_amount_policy']


Closed brands

In [87]:
condition = (
    (soh['Closed_status'] == "Closed") & 
    (~soh['location_catergory'].isin(["Leftover","Damage"] )) 
    )
soh.loc[condition, 'additional_provision'] =    (soh["NETTOTAL_COST"] * closed_percentage) - soh['provision_amount_policy']

Brand specific provisions

In [88]:
if brand_specific_provision:
    soh['additional_provision'] = np.where(
        soh['Std Brand'].isin(brand_specific_provision.keys()),
        soh['NETTOTAL_COST'] * soh['Std Brand'].map(brand_specific_provision) - soh['provision_amount_policy'],
        soh['additional_provision']
    )

removing the consginement and gurantee margin provision

In [89]:
soh.loc[
    soh['Model'].isin([
        'Consignment',
        'Guaranteed Margin',
        'Buying Pull - Mango'
    ]),
    ['provision_amount_policy', 'provision_%_policy', 'Continuity_factor','additional_provision']
] = 0

soh['provision_amount_policy'] = soh['provision_amount_policy'].fillna(0)
soh['additional_provision'] = soh['additional_provision'].fillna(0)


soh['Total Provision'] = soh['provision_amount_policy'] + soh['additional_provision']

In [90]:
soh.to_csv(r"Output/aging_provision.csv")

Summary

In [91]:
summary = soh.groupby(by='Std Brand')[["NETTOTAL_COST",'provision_amount_policy','additional_provision','Total Provision']].sum()
summary['coverage'] = summary['Total Provision'] / summary['NETTOTAL_COST']


Combinations

In [92]:
soh_with_combinations = soh.merge(combinations, on=['Std Brand','LOCATION'], how='left')
soh_with_combinations[['s1', 's2', 's3', 's4']]= soh_with_combinations[['s1', 's2', 's3', 's4']].fillna(0)
soh_with_combinations.to_excel(r"Output/aging_provision_combinations.xlsx", index=False)

Preparation of the entry

In [93]:
soh_with_combinations['Total Provision'].sum()

31003352.632795796

In [94]:
entry = soh_with_combinations.groupby(["s1","s2","s3","s4"])['Total Provision'].sum().reset_index().fillna(0)
entry['s5'] = 63002


In [95]:
entry['Total Provision'].sum()

31003352.6327958

In [96]:
entry2 = entry.copy()
entry2['Total Provision'] = entry2['Total Provision'] * -1
entry2['s5'] = 23993

In [97]:
completed_entry = pd.concat([entry, entry2], ignore_index=True)
completed_entry.rename(columns={'Total Provision': 'Dr/(CR)'}, inplace=True)
completed_entry = completed_entry[completed_entry['Dr/(CR)'] != 0]

In [98]:
completed_entry[['s1', 's2', 's3', 's4', 's5','Dr/(CR)']].to_csv(r"Output/completed_entry.csv", index=False)

difference entry

In [99]:
diff_table = soh_with_combinations.groupby(["s1","s2","s3","s4"])['Total Provision'].sum().reset_index().fillna(0).merge(existing_balances, on=['s1','s2','s3','s4'], how='outer').fillna(0)

In [100]:
diff_table['Dr/(CR)'] = (diff_table['Total Provision'] + diff_table['Closing balance'])*-1
diff_table.drop(['Closing balance','Total Provision'], inplace=True,axis=1)

In [101]:
diff_table['s5'] = 23993

In [102]:
diff_table2 = diff_table.copy()
diff_table2['Dr/(CR)'] = diff_table2['Dr/(CR)'] * -1
diff_table2['s5'] = 63002

In [103]:
diff_entry = pd.concat([diff_table, diff_table2], ignore_index=True)
#diff_entry.rename(columns={'Total Provision': 'Dr/(CR)'}, inplace=True)
diff_entry = diff_entry[diff_entry['Dr/(CR)'] != 0]

In [104]:
diff_entry[['s1', 's2', 's3', 's4', 's5','Dr/(CR)']].to_csv(r"Output/diff_entry.csv", index=False)

Analysis

In [105]:
damage_summary = soh_with_combinations[soh_with_combinations['location_catergory'] == 'Damage'].groupby('Std Brand')[['NETTOTAL_COST', 'Total Provision']].sum()
damage_summary['coverage'] = damage_summary['Total Provision'] / damage_summary['NETTOTAL_COST']
damage_summary

,NETTOTAL_COST,Total Provision,coverage
Std Brand,,,
ALO YOGA,"23,730.11","23,730.11",1.00
BERSHKA,"693,344.73","693,344.73",1.00
CLARKS,"64,129.72","64,129.72",1.00
DESIGUAL,"4,884.04",0.00,0.00
FG4 KIDS,"179,666.22","179,666.22",1.00
FG4 WOMEN,"260,973.89","260,973.89",1.00
FLYING TIGER,"17,745.63","17,745.63",1.00
GAP,"175,190.21","175,190.21",1.00
JACADI,"32,817.43",0.00,0.00


In [106]:
leftover_summary = soh_with_combinations[soh_with_combinations['location_catergory'] == 'Leftover'].groupby('Std Brand')[['NETTOTAL_COST', 'Total Provision']].sum()
leftover_summary['coverage'] = leftover_summary['Total Provision'] / leftover_summary['NETTOTAL_COST']
leftover_summary

,NETTOTAL_COST,Total Provision,coverage
Std Brand,,,
ALO YOGA,"22,284.23","11,142.11",0.50
ANOTAH,24.52,12.26,0.50
BERSHKA,"2,182,564.13","327,384.62",0.15
BOBBI BROWN,0.00,0.00,NaN
CLARKS,"7,080.84","3,540.42",0.50
DESIGUAL,"5,861.90",0.00,0.00
ELEMENT,"23,661.46","11,830.73",0.50
FG4 KIDS,"2,071,484.54","1,035,742.27",0.50
FG4 WOMEN,"648,148.02","324,074.01",0.50


In [107]:
closed_summary = soh_with_combinations[soh_with_combinations['Closed_status'] == 'Closed'].groupby('Std Brand')[['NETTOTAL_COST', 'Total Provision']].sum()
closed_summary['coverage'] = closed_summary['Total Provision'] / closed_summary['NETTOTAL_COST']
closed_summary

,NETTOTAL_COST,Total Provision,coverage
Std Brand,,,
AL MALKI,0.00,0.00,NaN
ALO YOGA,"46,014.34","34,872.23",0.76
ANOTAH,24.52,12.26,0.50
BOBBI BROWN,0.00,0.00,NaN
CLARKS,"71,210.56","67,670.14",0.95
DESIGUAL,"10,745.94",0.00,0.00
ELEMENT,"23,661.46","11,830.73",0.50
FG4 KIDS,"2,263,007.61","1,221,336.92",0.54
FG4 WOMEN,"909,061.79","585,017.84",0.64


checks

In [108]:
# verify the original seasons in the std buckets 
check_buckets = soh[['season_bucket','std_season' ]].drop_duplicates()

In [109]:
# verify the seasons in the correct buckets 
check_season = soh[['std_season', original_season]].drop_duplicates()

In [110]:
#missing combinations in output
missing_combinations = soh_with_combinations[(soh_with_combinations['NETTOTAL_COST'] != 0)&(soh_with_combinations['s1'].isna())] 

In [111]:
# 1. Check for missing values in key columns
missing_in_soh = soh[soh[['Std Brand', 'LOCATION_NAME', 'NETTOTAL_COST', 'std_season']].isnull().any(axis=1)]
print("Missing values in SOH key columns:", missing_in_soh.shape[0])


# 3. Check for duplicates in mapping [original brand name]
duplicates_mapping = mapping.duplicated(subset=['GROUP_NAME'], keep=False)
print("Duplicate original brand names in mapping:", mapping[duplicates_mapping].shape[0])

# 4. Check for missing standard brands in mapping and SOH
missing_std_brands_in_mapping = set(soh['Std Brand']) - set(mapping['Std Brand'])
print("Std Brands in SOH missing from mapping:", missing_std_brands_in_mapping)

missing_std_brands_in_soh = set(mapping['Std Brand']) - set(soh['Std Brand'])
print("Std Brands in mapping missing from SOH:", missing_std_brands_in_soh)

# 5. Check for garbage/unknown seasons in std_season
garbage_seasons = soh[soh['std_season'].isin(['Unknown', 'Old-', 'AW97'])]
print("Rows with garbage/unknown std_season:", garbage_seasons.shape[0])

# 6. Check for missing in combinations merge
missing_comb_rows = soh_with_combinations[soh_with_combinations['s1'].isnull()]
print("Rows missing combination mapping:", missing_comb_rows.shape[0])


Missing values in SOH key columns: 18
Duplicate original brand names in mapping: 0
Std Brands in SOH missing from mapping: {nan}
Std Brands in mapping missing from SOH: {'MONSOON ACCESSORIZE', 'IPEKYOL', "WOMEN'S SECRET", 'MACHKA', 'ESTEE LAUDER', 'PEDRO', 'MARIE FRANCE', 'ALDO & ALDO ACCS', 'ADL', 'CHARLES & KEITH', 'LA VIE EN ROSE', 'NINE WEST', 'OXXO', 'ZIDDY KIDS', 'FLORMAR', 'TWIST', 'SPRING', 'SPRINGFIELDS', 'LASENZA'}
Rows with garbage/unknown std_season: 1195
Rows missing combination mapping: 0


In [112]:
soh_with_combinations[soh_with_combinations[original_season].isna() | (soh_with_combinations[original_season] == '')].groupby('Std Brand')['NETTOTAL_COST'].sum()

Std Brand
BERSHKA            -582,951.04
Decathlon        -1,863,420.14
LEFTIES          -1,507,026.93
MASSIMO DUTTI     1,728,736.99
OYSHO               258,426.76
PULL AND BEAR     1,587,360.98
STRADIVARIOUS      -225,606.66
ZARA            -14,801,798.51
ZARA HOME           102,701.28
Name: NETTOTAL_COST, dtype: float64

In [113]:
completed_entry[completed_entry['s5']==23993]['Dr/(CR)'].sum(), existing_balances.iloc[:,-1].sum(),diff_entry[diff_entry['s5']==23993]['Dr/(CR)'].sum()

(-31003352.6327958, -34043425.849999994, 3040073.2172042)

In [114]:
completed_entry[completed_entry['s5']==23993]['Dr/(CR)'].sum(), existing_balances.iloc[:,-1].sum(),diff_entry[diff_entry['s5']==23993]['Dr/(CR)'].sum()

(-31003352.6327958, -34043425.849999994, 3040073.2172042)

In [115]:
entry_check = existing_balances.iloc[:,-1].sum()+ diff_entry[diff_entry['s5']==23993]['Dr/(CR)'].sum() - completed_entry[completed_entry['s5']==23993]['Dr/(CR)'].sum()

In [116]:
print(f"{entry_check=:,.2f}")

entry_check=0.00
